In [2]:
# Read the data from UIC mapping file, and get 2 columns: UICID and Phase, mapping it into dictionary
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Reading the file, store it in a variable as "dataset_prep_1"
# Command used: pd.read_csv
dataset_prep_1 = pd.read_csv("E:\\Recurrent Neural Network Workflow\\UICmapping.csv",encoding = 'latin1')


# Getting the 2 column by name
# Command used: df["column_name"]
UIC_ID_col = dataset_prep_1["#SampleID"]
UIC_Phase_col = dataset_prep_1["Phase"]

# Using the zip command to map 2 of the column into a "mapping_dict"
# Command used: dict(zip(column_1, column_2))
mapping_dict = dict(zip(UIC_ID_col, UIC_Phase_col))

# Print out and check out the result:
print(UIC_ID_col)
print(UIC_Phase_col)
print(mapping_dict)

0      UIC01
1      UIC02
2      UIC03
3      UIC04
4      UIC05
       ...  
127    UIC95
128    UIC96
129    UIC97
130    UIC98
131    UIC99
Name: #SampleID, Length: 132, dtype: object
0      Phase.1
1      Phase.1
2      Phase.1
3      Phase.1
4      Phase.1
        ...   
127    Phase.2
128    Phase.3
129    Phase.3
130    Phase.4
131    Phase.4
Name: Phase, Length: 132, dtype: object
{'UIC01': 'Phase.1', 'UIC02': 'Phase.1', 'UIC03': 'Phase.1', 'UIC04': 'Phase.1', 'UIC05': 'Phase.1', 'UIC06': 'Phase.1', 'UIC07': 'Phase.1', 'UIC08': 'Phase.1', 'UIC09': 'Phase.1', 'UIC10': 'Phase.2', 'UIC100': 'Phase.5', 'UIC101': 'Phase.5', 'UIC102': 'Phase.2', 'UIC103': 'Phase.2', 'UIC104': 'Phase.3', 'UIC105': 'Phase.3', 'UIC106': 'Phase.4', 'UIC107': 'Phase.4', 'UIC108': 'Phase.5', 'UIC109': 'Phase.5', 'UIC11': 'Phase.2', 'UIC110': 'Phase.2', 'UIC111': 'Phase.2', 'UIC112': 'Phase.3', 'UIC113': 'Phase.3', 'UIC114': 'Phase.4', 'UIC115': 'Phase.4', 'UIC116': 'Phase.5', 'UIC117': 'Phase.5', 'UIC118':

In [1]:
# Using the data of the mapping dict, transfer the data into the row title of the second set of data
# Converting all UIC_ID into seperate phase and sort it by index, output the new csv file

# Reading file 2 and store it as dataset_prep_2
dataset_prep_2 = pd.read_csv("E:\\Recurrent Neural Network Workflow\\BIOMtable.csv",encoding = 'latin1')

# Create a copy of dataframe dataset_prep_2
# Command used: dataframe.copy()
df_copy = dataset_prep_2.copy()


# Renaming the dataframe columns, mapping column UIC_ID to corresponding phase in column Phase
# Command used: dataframe.rename()
df_copy.rename(columns=mapping_dict, inplace=True)

# Sorting dataframe by Phase
# Command used: dataframe.sort_index(axis = 1)
df_copy = df_copy.sort_index(axis=1)

# Moving the column has microbiome name into the beginning of the dataframe after sorting
# Using pop to delete the column, then using insert to re-insert the column into the start
# Command used: dataframe.pop(column_to_move), dataframe.insert(column_to_add)
column_to_move = 'Unnamed: 0'
new_position = 0 
column_index = df_copy.pop(column_to_move)
df_copy.insert(new_position, column_to_move, column_index)

# Delete all the UIC_ID column that doesnt have phase mapping to it
string_to_exclude = 'UIC'
df_filtered = df_copy.drop(columns=[col for col in df_copy.columns if string_to_exclude in col])

# Create a variable that calculate the sum of row in the dataframe
taxa_sums = df_filtered.sum(axis=1)

# Edit the dataframe, removing all single OTS
df_filtered = df_filtered[taxa_sums > 1]

# Exporting file into a new csv, not including the index
# Command used: dataframe.to_csv('name.csv', index = False)
df_filtered.to_csv('New_mapping_file.csv', index=False)

# Print out the dataframe:
print(df_filtered)

NameError: name 'pd' is not defined

In [ ]:
# Attirbute of the data:
# 1. Uneven number of columns for timeframe
# 2. Train data with multiple column of timeframe
# 3. Phase 1, phase 2, phase 3 is in season, phase 4 is competition, phase 5 and phase 6 is off-season

In [7]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Set seed for numpy and tensorflow for reproduction
np.random.seed(8)
tf.random.set_seed(8)

dataset_train = pd.read_csv("E:\\Recurrent Neural Network Workflow\\New_mapping_file.csv")
df = dataset_train.copy()
#print(df)

# Getting total of each Phase for easier training:
# Total phase 1: 
df['Phase 1 Total'] = df.iloc[:, 1:10].sum(axis=1)
# Total phase 2:
df['Phase 2 Total'] = df.iloc[:, 10:50].sum(axis=1)
# Total phase 3:
df['Phase 3 Total'] = df.iloc[:, 50:91].sum(axis=1)
# Total phase 4:
df['Phase 4 Total'] = df.iloc[:,91:105].sum(axis=1)
# Total phase 5:
df['Phase 5 Total'] = df.iloc[:,105:119].sum(axis=1)
# Total phase 6:
df['Phase 6 Total'] = df.iloc[:,119:133].sum(axis=1)

# Getting all phase into one new dataframe and output
new_df_need_to_transpose = pd.concat([df['Phase 1 Total'] , df['Phase 2 Total'] , df['Phase 3 Total'] , df['Phase 4 Total'] , df['Phase 5 Total'], df['Phase 6 Total']], axis=1)
new_df = new_df_need_to_transpose.transpose()
print(new_df)

# Getting 70% of data for training, output random sample size, as well as set seed
sample_size = int(0.7 * len(new_df))
sampled_data = new_df.sample(n=sample_size, random_state = 123)
print(sampled_data)
'''
# Convert data into numpy
data = new_df.values
num_otus, num_phases = data.shape

time_steps = 2
num_features = data.shape[1]

# Prepare data for RNN
x, y = [], []

for i in range(len(data) - time_steps):
    x.append(data[i:i+time_steps])
    y.append(data[i+time_steps])

x = np.array(x)
y = np.array(y)

# Define the RNN model
model = Sequential()
model.add(LSTM(units=64, input_shape=(time_steps, num_features)))
model.add(Dense(units=num_features))

# Compile the model
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_absolute_error"])

# Train the model
history = model.fit(x, y, epochs=100, batch_size= 32)  

# Make predictions
predictions = model.predict(x)

# Print the predictions
print(predictions)
'''

               0      1      2      3      4      5      6      7      8      \
Phase 1 Total      0     44      0      6      0      0      0      0      0   
Phase 2 Total      0    338      3      4      0      1      1      0      2   
Phase 3 Total      0    428      0     21      0      1      0      2      3   
Phase 4 Total      0     98      1      0      0      0      0      1      0   
Phase 5 Total      0    155      1      0      0      0      0      2      0   
Phase 6 Total      3     99      0      0      3      0      3      1      0   

               9      ...  12296  12297  12298  12299  12300  12301  12302  \
Phase 1 Total      0  ...    606   1156   1660   1206     19   5848   1185   
Phase 2 Total      3  ...    623  13774  11821  10929   1206  40513  15318   
Phase 3 Total      0  ...    804  10018   8995   7373    280  36422   6726   
Phase 4 Total      5  ...      0   8802   7675   8673    747  22942   9995   
Phase 5 Total      1  ...      0  16150   5109  1

C:\Users\Tran Xuan Duc\AppData\Local\Temp\ipykernel_7832\961401563.py:12: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_train = pd.read_csv("E:\\Recurrent Neural Network Workflow\\New_mapping_file.csv")


'\n# Convert data into numpy\ndata = new_df.values\nnum_otus, num_phases = data.shape\n\ntime_steps = 2\nnum_features = data.shape[1]\n\n# Prepare data for RNN\nx, y = [], []\n\nfor i in range(len(data) - time_steps):\n    x.append(data[i:i+time_steps])\n    y.append(data[i+time_steps])\n\nx = np.array(x)\ny = np.array(y)\n\n# Define the RNN model\nmodel = Sequential()\nmodel.add(LSTM(units=64, input_shape=(time_steps, num_features)))\nmodel.add(Dense(units=num_features))\n\n# Compile the model\nmodel.compile(loss="mean_squared_error", optimizer="adam", metrics=["mean_absolute_error"])\n\n# Train the model\nhistory = model.fit(x, y, epochs=100, batch_size= 32)  \n\n# Make predictions\npredictions = model.predict(x)\n\n# Print the predictions\nprint(predictions)\n'